In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# /Users/chriswesterman/Library/Jupyter/nbextensions/snippets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import beatthebookies.data as btb
import sqlite3

pd.set_option('display.max_columns', 500)


# Exploratory notebook

In [11]:
path = '../data/fifarank.csv'

In [12]:
df = pd.read_csv(path)

In [16]:
df.Team.unique()

array(['Arsenal', 'Manchester United', 'Chelsea', 'Liverpool',
       'Newcastle United', 'Middlesbrough', 'Fulham', 'Tottenham Hotspur',
       'Manchester City', 'Birmingham City', 'Aston Villa',
       'Charlton Athletic', 'Bolton Wanderers', 'Blackburn Rovers',
       'Southampton', 'Portsmouth', 'Everton', 'West Bromwich',
       'Norwich City', 'Crystal Palace', 'West Ham United',
       'Wigan Athletic', 'Sunderland', 'Sheffield United', 'Reading FC',
       'Watford', 'Chelsea FC', 'Arsenal FC', 'Derby County', 'Hull City',
       'Stoke City', 'Burnley', 'Wolverhampton Wanderers',
       'West Bromwich Albion', 'Blackpool', 'Queens Park Rangers',
       'Swansea City', 'Reading', 'Cardiff City', 'Leicester City',
       'Bournemouth', 'Brighton & Hove Albion', 'Huddersfield Town',
       'AFC Bournemouth'], dtype=object)

In [5]:
matches = btb.get_data()

get_data 9.0


In [7]:
seasons = ['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
for year in seasons:
    print(f'20{year}/20{str(int(year)+1).zfill(2)}')

2006/2007
2007/2008
2008/2009
2009/2010
2010/2011
2011/2012
2012/2013
2013/2014
2014/2015
2015/2016
2016/2017
2017/2018
2018/2019
2019/2020
2020/2021


In [6]:
matches.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_home_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,home_t_total_goals,home_t_total_goals_against,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_total_goals,away_t_total_goals_against,away_t_total_wins,away_t_total_losses,away_t_total_draws,bups_x,bupp_x,ccp_x,ccc_x,ccs_x,dp_x,da_x,dtw_x,bups_y,bupp_y,ccp_y,ccc_y,ccs_y,dp_y,da_y,dtw_y
375,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,30,5,8,5,13,34,3,13,2,56,55,10,13,14,39,64,9,21,7,43,40,39,33,63,52,58,59,45,55,46,54,47,38,39,51
376,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,29,5,5,8,32,17,11,2,5,58,52,12,12,13,67,35,23,3,11,67,36,41,34,44,39,41,46,63,60,47,64,46,58,65,55
377,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,11,11,3,4,13,37,1,14,3,61,36,19,7,11,27,72,3,26,8,59,30,28,44,46,51,44,52,63,54,60,48,38,35,44,54
378,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,21,10,5,3,19,24,5,7,6,55,40,17,11,9,38,47,11,17,9,52,46,42,54,30,50,65,44,61,67,67,69,50,36,34,41
379,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,8,11,2,5,21,30,6,8,4,46,34,18,10,9,44,64,11,17,9,38,44,49,44,40,54,53,56,47,47,49,45,47,48,50,62


In [ ]:
sql_path = '../data/'
database = sql_path + 'database.sqlite'
conn = sqlite3.connect(database)

In [ ]:
df = pd.read_sql("""SELECT m.id,
                        m.season, m.stage, m.date,
                        ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal,
                        m.away_team_goal, m.home_team_api_id as home_api, m.away_team_api_id as away_api
                        FROM Match as m
                        LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                        LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        WHERE league_id = :league AND season != :season
                        ;""", conn, params={"league":1729, "season":'2008/2009'})

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
stats2 = pd.read_sql("""Select * from Team_Attributes""", conn)
stats2.head()

In [ ]:
stats2.buildUpPlayPositionClass.unique()

In [ ]:
h_stats = pd.read_sql("""Select * from Team_Attributes""", conn)
a_stats = pd.read_sql("""Select * from Team_Attributes""", conn)
h_stats['date'] = pd.to_datetime(h_stats['date'])
h_stats['year'] = pd.DatetimeIndex(h_stats['date']).year
season_maps = {2010: '2009/2010', 2011: '2010/2011', 2012:'2011/2012',
             2013:'2013/2014', 2014: '2014/2015', 2015:'2015/2016'}
h_stats['season'] = h_stats['year'].map(season_maps)
h_stats = h_stats[['season', 'team_api_id', 'buildUpPlaySpeed']]

a_stats['date'] = pd.to_datetime(a_stats['date'])
a_stats['year'] = pd.DatetimeIndex(a_stats['date']).year
season_maps = {2010: '2009/2010', 2011: '2010/2011', 2012:'2011/2012',
             2013:'2013/2014', 2014: '2014/2015', 2015:'2015/2016'}
a_stats['season'] = a_stats['year'].map(season_maps)
a_stats = a_stats[['season', 'team_api_id', 'buildUpPlaySpeed']]


In [ ]:
h_stats.head()

In [ ]:
h_stats[h_stats['team_api_id'] == 10260 ]

In [ ]:
a_stats[a_stats['team_api_id'] == 8655]

In [ ]:
df = pd.merge(df, h_stats, how='left', left_on=['season', 'home_api'], right_on=['season', 'team_api_id'])

In [ ]:
df = df.rename(columns={'buildUpPlaySpeed': 'home speed'})

In [ ]:
df.head()

In [ ]:
df = pd.merge(df, a_stats, how='left', left_on=['season', 'away_api'], right_on=['season', 'team_api_id'])

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df[df['home_api'] == 10260]

In [ ]:
columns = [['buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing', 'chanceCreationPassing', 
            'chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth']]

In [ ]:
stats.date.value_counts()

In [ ]:
mock_score = pd.DataFrame([['A', 1, 90], ['A', 2, 80], ['A',3,100], ['B', 1, 95], ['B',2,85]], columns = ['year', 'team', 'stat'])

In [ ]:
mock = pd.DataFrame([['A',1,1],
                    ['A',2,2],
                    ['A',1,3],
                    ['A',2,4],
                    ['A',1,5],
                    ['B',1,6],
                    ['B',2,7],
                    ['B',1,8],
                    ['B',2,9],
                    ['B',1,10]], columns=['year','team','score'])

In [ ]:
mock, mock_score

In [ ]:
test = pd.merge(mock, mock_score, on=['year', 'team'])

In [ ]:
test